# Post-Processing Amazon Textract with Location-Aware Transformers

# Part 1: Introduction and Data Collection

> *This notebook works well with the `Data Science 3.0 (Python 3)` kernel on SageMaker Studio*

Human-readable documents are more than just 1D sequences of text: Many have **complex layout and formatting structures** (like headers and footers, form fields, tables, paragraphs and multi-section layouts...) that are important for a wide range of information extraction tasks - and challenge the idea that pages always have one "correct reading order" at all.

**Multi-modal, layout-aware language models** - like [**DocFormer** (2021, Appalaraju et al at AWS AI)](https://arxiv.org/abs/2106.11539), [**LayoutXLM** (2021, Xu et al)](https://arxiv.org/abs/2104.08836), and the LayoutLM series [v1](https://arxiv.org/abs/1912.13318), [v2](https://arxiv.org/abs/2012.14740), [v3](https://arxiv.org/abs/2204.08387) - augment traditional (sequence-based) text processing models to add features representing the **absolute positions of words/tokens on the page**: And other modalities too, like a pixel image representation of the source page.

Architectures like these can **significantly improve accuracy** (over plain text methods or position-based templating), for information extraction tasks where *both* the visual layout and the text content are useful.

> ⚠️ **Note:** You should be aware that several fully-managed document AI service offerings on AWS, are already capable of similar kinds of multi-modal document understanding under the hood: Particularly [Amazon Textract Queries](https://aws.amazon.com/about-aws/whats-new/2022/04/amazon-textract-launches-queries-feature-analyze-document-api/) and [Amazon Comprehend native document processing](https://aws.amazon.com/about-aws/whats-new/2021/09/amazon-comprehend-extract-entities-native-format/).
>
> While these AI services should usually be the first choice where available, custom SageMaker-based modelling as shown here may still be of interest in some cases: For example if you'd like to customize the model architeture or pre-training, or use a particular feature+language combination that's not currently available as a managed service.

This multi-modal approach is useful for post-processing OCR results from services like [Amazon Textract](https://aws.amazon.com/textract/) (which returns both the detected text and the geometry of each word in the input document), and can be compatible with a broad range of downstream NLP tasks, like:

- "Self-supervised" pre-training on Textracted but otherwise unlabelled documents
- Classifying documents, pages, or individual lines/sequences
- Extracting entities (e.g. via token/word classification)
- Extractive question answering and other span extraction tasks
- "Translation", generative question answering or other sequence generation

In this sample we'll demonstrate applying open-source models in this family (including the multi-lingual **[LayoutXLM](https://huggingface.co/docs/transformers/model_doc/layoutxlm)** and English-optimized **LayoutLM [v1](https://huggingface.co/docs/transformers/model_doc/layoutlm)/[v2](https://huggingface.co/docs/transformers/model_doc/layoutlmv2)**) to an example use case where Amazon Textract's [built-in functionality](https://aws.amazon.com/textract/features/) for extracting key-value "Forms" data and structured "Tables" data helps somewhat... But would require further post-processing for usable results.

> ⚠️ **Note:** Although model implementations in the [Hugging Face Transformers library](https://huggingface.co/docs/transformers/index) used by this sample are generally published under the permissive Apache-2.0 license [as detailed on their GitHub repository](https://github.com/huggingface/transformers/blob/main/LICENSE), you should be aware that **license terms for pre-trained model weights** published on the [Hugging Face Model Hub](https://huggingface.co/models) may vary. In particular, there is [a difference](https://github.com/microsoft/unilm/issues/352) in commercial-use licensing between pre-trained LayoutLMv1 and v2/v3/XLM weights.

This first notebook will focus on preparing and annotating data, before we move on to training, deploying, and integrating models in later notebooks.

## Data prep overview (⚙️ bringing your own data)

This notebook covers 3 main data preparation steps for training multi-modal document understanding models like these:

1. Processing raw documents with **Amazon Textract** to extract position-linked text information.
1. **Cleaning** document images ready for human annotation (and, for some models, preparing standard-sized page images).
1. **Iteratively annotating** documents to build a dataset.

We'll try to keep this process **modular** to make it easy to adapt for your own datasets and iterate as your corpus grows.

For this purpose, we'll use [JSONLines](https://jsonlines.org/) formatted [augmented manifest files](https://docs.aws.amazon.com/sagemaker/latest/dg/augmented-manifest.html#augmented-manifest-format) as a shared integration point: Starting out with a simple document-level list of raw files, and gradually building up to an annotated, page-level manifest with records something like the below:

```json
{"raw-ref": "s3://doc-example-bucket/raw/mydoc.pdf", "textract-ref": "s3://doc-example-bucket/tex/mydoc.pdf.json", "page-num": 2, "source-ref": "s3://doc-example-bucket/img/mydoc-0002.png", "labels": {...}}
{"raw-ref": "s3://doc-example-bucket/raw/mydoc.pdf", "textract-ref": "s3://doc-example-bucket/tex/mydoc.pdf.json", "page-num": 3, "source-ref": "s3://doc-example-bucket/img/mydoc-0003.png", "labels": {...}}
```

If your setup deviates at any step (for example, already having Textract results for your documents) or you want to modify the data, then so long as you're able to generate the manifest file output for each step, you should be able to continue the flow. 

## Getting started: Dependencies and configuration

First there are some additional libraries we need to install that aren't present in the SageMaker kernel environments by default:

- [Amazon Textract Response Parser](https://github.com/aws-samples/amazon-textract-response-parser) is a helper for interpreting and navigating Amazon Textract result JSON.
- The [SageMaker Studio Image Build CLI](https://github.com/aws-samples/sagemaker-studio-image-build-cli) is a tool for building customized container images and pushing to [Amazon ECR](https://aws.amazon.com/ecr/).
- [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/) (installed by default in SageMaker already) must be version >=2.87 for some of the Hugging Face container versions used in notebook 2.

In [85]:
!pip install amazon-textract-response-parser \
    sagemaker-studio-image-build \
    "sagemaker>=2.87,<3"


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


With the requried libraries installed, we're ready to import dependencies and set up some basic configuration including which [Amazon S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html) and folder/prefix data will be uploaded to.

In [86]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
from logging import getLogger
import os
import random
import re
import shutil
import time
from zipfile import ZipFile

# External Dependencies:
import boto3  # AWS SDK for Python
from IPython import display  # To display rich content in notebook
import pandas as pd  # For tabular data analysis
import sagemaker  # High-level SDK for SageMaker
from tqdm.notebook import tqdm  # Progress bars

# Local Dependencies:
import util

logger = getLogger()

#ucket_name = sagemaker.Session().default_bucket()
#bucket_prefix = "textract-transformers/"

# Use our own bucket loaded with training images 2023-05-23
bucket_name = 'bt-digital-bt-labs-internal-200'
bucket_prefix = "DynamicTableParser/4_TrainingData/"

print()
print ("Bucket Name: " + bucket_name)
print ("Bucket Prefix: " + bucket_prefix)
print()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Bucket Name: bt-digital-bt-labs-internal-200
Bucket Prefix: DynamicTableParser/4_TrainingData/



For some sections below we'll need to reference resources created by the **[AWS CloudFormation](https://aws.amazon.com/cloudformation/) solution stack** you spun up earlier. If you didn't do this step yet, refer to the [README.md](../README.md) in the top level of the repository for instructions.

The solution stack stores these useful variables in [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html) and below we use the `util.project` utility module ([util/project.py](util/project.py)) to fetch them. This is a transferable pattern you can use to connect from data science notebooks to deployed ML project resources in the cloud by project name/ID.

▶️ **Check** in the [CloudFormation Console](https://console.aws.amazon.com/cloudformation/home?#/stacks) that the `ProjectId` parameter for your OCR Pipeline Stack matches the default `ocr-transformers-demo` value below: Otherwise change the code below to match.

> ⚠️ If you get an **AccessDeniedException** (ClientError) below, it's likely your [SageMaker execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) doesn't have the required `ssm:GetParameters` permission to look up the OCR pipeline stack parameters.
>
> To fix this, you can click your execution role in the [IAM Roles Console](https://console.aws.amazon.com/iamv2/home#/roles) and use the **Attach policies** button to attach the `PipelineDataSciencePolicy` created by the stack.

In [87]:
try:
    config = util.project.init("ocr-transformers-demo")
    print(config)
except Exception as e:
    try:
        print(f"Your SageMaker execution role is: {sagemaker.get_execution_role()}")
    except Exception:
        print("Couldn't look up your SageMaker execution role")
    raise e

<util.project.ProjectSession(
  project_id=ocr-transformers-demo,
  a2i_review_flow_arn_param=/ocr-transformers-demo/config/HumanReviewFlowArn,
  entity_config_param=/ocr-transformers-demo/config/EntityConfiguration,
  sagemaker_endpoint_name_param=/ocr-transformers-demo/config/SageMakerEndpointName,
  thumbnail_endpoint_name_param=/ocr-transformers-demo/config/ThumbnailEndpointName,
  a2i_execution_role_arn=arn:aws:iam::015943506230:role/OCRPipelineDemo-ProcessingPipelineReviewStepProces-M6UD1K31HQ0Q,
  pipeline_input_bucket_name=ocrpipelinedemo-pipelineinputbucket350ea1ae-1ffox60d5xk2z,
  model_callback_topic_arn=arn:aws:sns:us-east-1:015943506230:OCRPipelineDemo-ProcessingPipelineEnrichmentStepNLPEnrichmentModelSageMakerAsyncNLPEnrichmentModelE62625A1-WxGfSPiKRll3,
  model_results_bucket=ocrpipelinedemo-processingpipelineenrichedresults-bt6ehtmayta3,
  pipeline_sfn_arn=arn:aws:states:us-east-1:015943506230:stateMachine:ProcessingPipelinePipelineStateMachineC698BCB6-OfF7h2R7eWfO,
  p

## Upload the raw documents

For our example, we'll be exploring (a recent quarter's snapshot of) the [Credit card agreements database](https://www.consumerfinance.gov/credit-cards/agreements/) published by the United States' [Consumer Finance Protection Bureau](https://www.consumerfinance.gov/). This dataset includes specimen credit card agreement documents from providers across the US, and is interesting for our purposes because the documents are:

- **Diverse** in formatting, as various providers present the required information in different ways
- **Representative of commercial** documents - rather than, for example, academic papers which might have quite different tone and structure
- **Complex** in structure, with common data points in theory (e.g. interest rates, fees, etc) - but a lot of nuances and differences between documents in practice.

The sample dataset (approx. 900MB uncompressed) is published as an archive file (approx. 750MB) which we'll need to extract for the raw PDFs. Since it's a reasonable size, we can perform the extraction here in SageMaker Studio to also have local copies of the raw files to inspect.

> ▶️ Skip down to the next cell for guidance on bringing custom datasets (or if you've run this extraction already). You may find it useful to run this sample through once with the example dataset first - to understand how it works end-to-end.

In [88]:
%%time
os.makedirs("data/raw", exist_ok=True)

# Fetch the example data:
!wget -O data/CC_Agreements.zip https://files.consumerfinance.gov/a/assets/Credit_Card_Agreements_2020_Q4.zip

# Extract the file:
print("Extracting...")
shutil.rmtree("data/raw")
with ZipFile("data/CC_Agreements.zip", "r") as fzip:
    fzip.extractall("data/raw")

# Clean up unneeded files and remap if the folder became nested:
# (This is written specific to our sample data zip, but is unlikely to break most custom data)
original_root_items = os.listdir("data/raw")
if "__MACOSX" in original_root_items:
    shutil.rmtree("data/raw/__MACOSX")
if len(original_root_items) < 4:
    try:
        folder = next(f for f in original_root_items if f.startswith("Credit_Card_Agreements"))
        print(f"De-nesting folder '{folder}'...")
        for sub in os.listdir(f"data/raw/{folder}"):
            shutil.move(f"data/raw/{folder}/{sub}", f"data/raw/{sub}")
            time.sleep(0.1)  # (Saw a FileNotFound error during renames one time in SMStudio)
        os.rmdir(f"data/raw/{folder}")
    except StopIteration:
        pass

print("Done!")

--2023-05-24 03:27:51--  https://files.consumerfinance.gov/a/assets/Credit_Card_Agreements_2020_Q4.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 13.32.151.63, 13.32.151.58, 13.32.151.126, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|13.32.151.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764289046 (729M) [application/zip]
Saving to: ‘data/CC_Agreements.zip’

data/CC_Agreements. 100%[===================>] 728.88M  62.5MB/s    in 11s     

2023-05-24 03:28:02 (68.2 MB/s) - ‘data/CC_Agreements.zip’ saved [764289046/764289046]

Extracting...
De-nesting folder 'Credit_Card_Agreements_2020_Q4'...
Done!
CPU times: user 5 s, sys: 1.88 s, total: 6.88 s
Wall time: 2min 54s


⚙️ For custom datasets, your documents are likely already extracted.

- They'll need to be uploaded to **Amazon S3 in the same AWS Region** you're currently working in, in a bucket your [SageMaker execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has access to - in order to use our sample code to run them through Amazon Textract in bulk, prepare page thumbnail images, or annotate the documents.
- They should be under one **shared S3 prefix** - to allow generating all page thumbnail images in a single job, and because downstream processes expect a relative path to uniquely identify a document.
- You may like to copy (all or a small subset of) these docs into SageMaker under the `data/raw` folder - to preview and inspect documents through the process.

In [106]:
os.makedirs("data/raw", exist_ok=True)

# Configure your root S3 location for raw documents (note no trailing slash):
raw_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/raw"
print (raw_s3uri)

s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw


In [107]:
# The s3 sync command can upload folders from SageMaker to S3 (or download, swapping the args).
# For the example data, we extracted locally so will upload:
print(f"Downloading raw PDFs from {raw_s3uri}...")
!aws s3 sync --quiet {raw_s3uri} data/raw
print("Done")

Done


With the documents present on S3, we'll need an inventory of **all** documents relevant to the project, and will use a [JSONLines augmented manifest format](https://docs.aws.amazon.com/sagemaker/latest/dg/augmented-manifest.html#augmented-manifest-format) file to store this.

The cell below builds this by crawling all objects under `raw_s3uri` that look like valid images/documents:

In [92]:
raw_bucket_name, raw_prefix = util.s3.s3uri_to_bucket_and_key(raw_s3uri)
print (raw_bucket_name)
print (raw_prefix)

bt-digital-bt-labs-internal-200
DynamicTableParser/4_TrainingData/data/raw


In [93]:
s3 = boto3.resource("s3")
raw_bucket_name, raw_prefix = util.s3.s3uri_to_bucket_and_key(raw_s3uri)
print (raw_bucket_name)
print (raw_prefix)

valid_file_types = {"jpeg", "jpg", "pdf", "png", "tif", "tiff"}

n_files = 0
with open("data/raw-all.manifest.jsonl", "w") as f:
    # sorted() guarantees output order for reproducible sampling later:
    for obj in sorted(
        s3.Bucket(raw_bucket_name).objects.filter(Prefix=raw_prefix + "/"),
        key=lambda obj: obj.key,
    ):
        # Filter out any files you know shouldn't be counted:
        file_ext = obj.key.rpartition(".")[2].lower()
        if "/." in obj.key or file_ext not in valid_file_types:
            print(f"Skipping s3://{obj.bucket_name}/{obj.key}")
            continue

        # Save
        item = {"raw-ref": f"s3://{obj.bucket_name}/{obj.key}"}
        f.write(json.dumps(item)+"\n")
        n_files += 1

print(f"\nFound {n_files} valid files for OCR")

bt-digital-bt-labs-internal-200
DynamicTableParser/4_TrainingData/data/raw
Skipping s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/

Found 100 valid files for OCR


> ▶️ In [data/raw-all.manifest.jsonl](data/raw-all.manifest.jsonl) you should now see an alphabetized list of your source documents as stored on S3: With one record per line like `{"raw-ref": "s3://..."}`

**IF** you have (some or all of) the source documents downloaded to SageMaker in the `data/raw` folder, you can explore these through the file browser or  even pull them through to display inline here in the notebook:

In [95]:
# Read from docs manifest:
with open("data/raw-all.manifest.jsonl") as f:
    for ix, line in enumerate(f):
        # Select by index (could change this):
        if ix == 0:
            item = json.loads(line)
            # Map S3 URI to local data/raw file path:
            filepath = os.path.join("data/raw", item["raw-ref"].replace(raw_s3uri+"/", ""))
            break

print(f"Displaying: {filepath}")
display.IFrame(
    filepath,
    height="600",
    width="100%",
)

Displaying: data/raw/019112540_-_Juul_Labs_-_Francis_Howell_School_District-3.jpeg


## Defining the challenge

We have our source documents, so what will we try to extract about them?

There are many ways position-aware NLP models might be applied to OCR outputs: For example to generate structured summaries, provide translations, answer questions, or just classify the documents.

A common requirement in document analytics and process automation though, is to extract particular **'fields' of interest**: Known attributes expected to be present in all/most of the documents, which would be interesting to compare between them.

In this example we'll tackle this as an **entity detection** task **via word classification**:

- Defining a list of field/entity types of interest
- Classifying each `WORD` in the document to these types, using a Hugging Face [LayoutLMv2ForTokenClassification](https://huggingface.co/docs/transformers/model_doc/layoutlmv2#transformers.LayoutLMv2ForTokenClassification) model or similar
- ...And finally grouping individual words together (via simple rule-based post-processing) to detect the entities/fields

Some **benefits** of this approach are:

- Results are traceable all the way back to the detected word blocks from the OCR engine; rather than with a text generation method where the output of the model may not correspond 1:1 with raw text.
- Annotation effort is relatively minimal; since we only need to highlight the documents, rather than typing out custom corrections, answers, etc.

Some **drawbacks** are:

- Since it only tags detected words, this model will not be able to *intelligently correct OCR errors* or *standardize form* (e.g. of dates) like a text generation method could learn to.
- Since the ML component only extends to word classification, we're still relying on (usually good, helped by Amazon Textract) rule-based heuristics to group same-type words together to detect multi-word entities.

Below, we'll define the set of fields/entities to be detected and their configuration aspects. You can run `help(FieldConfiguration)` or refer to [util/postproc/config.py](util/postproc/config.py) for more details of the API.

> ⚠️ **Warning:** Although you may **edit** the configuration below (to build a custom dataset), you'll no longer be able to use the pre-annotated data sample we provide in `data/annotations` to accelerate model training (unless the classes are still defined in the same order, and labelled in a consistent way with the previous guidelines)

In [46]:
from util.postproc.config import FieldConfiguration

# For config API details, you can see the docs in the source file or run:
# help(FieldConfiguration)

fields = [
    # (To prevent human error, enter class_id=0 each time and update programmatically below)
    FieldConfiguration(0, "Agreement Effective Date", optional=True, select="first",
        annotation_guidance=(
            "<p>Avoid labeling extraneous dates which are not necessarily the effective date of "
            "the document: E.g. copyright dates/years, or other dates mentioned in text.</p> "
            "<p>Do not include unnecessary qualifiers e.g. 'from 2020/01/01'.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - Introductory", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Use this class (instead of the others) for <em>ANY</em> case where the rate is "
            "offered for a fixed introductory period - regardless of interest rate subtype e.g. "
            "balance transfers, purchases, etc.</p> "
            "<p>Include the term of the introductory period in cases where it's directly listed "
            "(e.g. '20.00% for the first 6 months'). Try to minimize/exclude extraneous "
            "information about the offer (e.g. '20.00% for the first 6 months after account "
            "opening').</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - Balance Transfers", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Use for interest rates which are specific to balance transfers.</p> "
            "<p>Avoid including extraneous information about the terms of balance transfers, or "
            "using for fixed-term introductory rates.</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - Cash Advances", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Use for interest rates which are specific to cash advances.</p> "
            "<p>Avoid including extraneous information about the terms of cash advances, or using "
            "for fixed-term introductory rates.</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - Purchases", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Use for interest rates which are specific to purchases.</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - Penalty", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Use for penalty interest rates applied under certain conditions.</p> "
            "<p><em>Exclude</em> include information about the conditions under which the penalty "
            "rate comes into effect: Only include the interest rate which will be applied.</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - General", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Use for interest rates which are general and not specifically tied to a "
            "particular transaction type e.g. purchases / balance transfers.</p> "
            "<p>Avoid using for fixed-term introductory rates.</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "APR - Other", optional=True, select="confidence",
        # TODO: Remove this class
        annotation_guidance=(
            "<p>Use only for interest rates which don't fall in to any other category (including "
            "general or introductory rates). You may not see any examples in the data.</p> "
            "<p>Avoid using for fixed-term introductory rates.</p> "
            "<p>'Prime rate + X%' mentions are acceptable and should be labeled.</p>"
        ),
    ),
    FieldConfiguration(0, "Fee - Annual", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Include cases where the document explicitly indicates no fee e.g. 'None'</p> "
            "<p>Avoid any introductory terms e.g. '$0 for the first 6 months' or extraneous "
            "words: Label only the standard fee.</p> "
            "<p>Label only the annual amount of the fee, in cases where other breakdowns are "
            "specified: E.g. '$120', not '$10 per month ($120 per year)'.</p> "
        ),
    ),
    FieldConfiguration(0, "Fee - Balance Transfer", optional=True, select="confidence",
        annotation_guidance=(
            # TODO: Review
            "<p>Try to be concise and exclude extra terms where not necessary</p>"
        ),
    ),
    FieldConfiguration(0, "Fee - Late Payment", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Label only the fee, not the circumstances in which it is payable.</p> "
            "<p>Limits e.g. 'Up to $25' are acceptable (don't just label '$25').</p> "
            "<p>Do <em>NOT</em> include non-specific mentions of pass-throgh costs (e.g. 'legal "
            "costs', 'reasonable expenses', etc.) incurred in the general collections process.</p>"
        ),
    ),
    FieldConfiguration(0, "Fee - Returned Payment", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Label only the fee, not the circumstances in which it is payable.</p> "
            "<p>Limits e.g. 'Up to $25' are acceptable (don't just label '$25').</p>"
        ),
    ),
    FieldConfiguration(0, "Fee - Foreign Transaction", optional=True, select="shortest",
        annotation_guidance=(
            "<p>Do <em>NOT</em> include explanations of how exchange rates are calculated or "
            "non-specific indications of margins between rates. <em>DO</em> include specific "
            "charges/margins with <em>brief</em> clarifying info where listed e.g. '3% of the US "
            "dollar amount'.</p>"
        ),
    ),
    FieldConfiguration(0, "Fee - Other", ignore=True,
        annotation_guidance=(
            "<p>Common examples include: Minimum interest charge, cash advance fees, and "
            "overlimit fees.</p> "
            "<p>Do <em>NOT</em> include fixed-term introductory rates for fees (e.g. '$0 during "
            "the first year. After the first year...') - only the standard fees</p> "
            "<p><em>DO</em> include qualifying information on the amount and limits of the fee, "
            "e.g. '$5 or 5% of the amount of each transaction, whichever is the greater'.</p> "
            "<p>Do <em>NOT</em> include general information on the nature of the fee and "
            "circumstances under which it is applied: E.g. 'Cash advance fee' or 'If the amount "
            "of interest payable is...'</p>"
        ),
    ),
    FieldConfiguration(0, "Card Name",
        annotation_guidance=(
            "<p>Label instances of the brand name of specific card(s) offered by the provider "
            "under the agreement, e.g. 'Rewards Platinum Card'</p> "
            "<p>Include the ' Card' suffix where available, but also annotate instances without "
            "such as 'Rewards Platinum'</p> "
            "<p><em>Avoid</em> including the Provider Name (use the separate class for this) e.g. "
            "'AnyCompany Rewards Card' unless it's been substantially modified/abbreviated for "
            "the card name (e.g. 'AnyCo Rewards Card') or the company name is different from the "
            "Credit card provider (e.g. AnyBank offering a store credit card for AnyCompany)</p> "
            "<p>Do <em>NOT</em> include fixed-term introductory rates for fees (e.g. '$0 during "
            "the first year. After the first year...') - only the standard fees</p> "
            "<p><em>Avoid</em> labeling generic payment provider names e.g. 'VISA card' or "
            "'Mastercard', except in contexts where the provider clearly uses them as the brand "
            "name for the offered card (e.g. 'VISA Card' from 'AnyCompany VISA Card'.</p>"
        ),
    ),
    FieldConfiguration(0, "Provider Address", optional=True, select="confidence",
        annotation_guidance=(
            "<p>Include department or 'attn:' lines where present (but not Provider Name where "
            "used at the start of an address e.g. 'AnyCompany; 100 Main Street...').</p> "
            "<p>Include zip/postcode where present.</p> "
            "<p><em>Avoid</em> labeling addresses for non-provider entities, such as watchdogs, "
            "market regulators, or independent agencies.</p>"
        ),
    ),
    FieldConfiguration(0, "Provider Name", select="longest",
        annotation_guidance=(
            "<p>Label the name of the card provider: Including abbreviated mentions.</p>"
        ),
    ),
    FieldConfiguration(0, "Min Payment Calculation", ignore=True,
        annotation_guidance=(
            "<p>Label clauses describing how the minimum payment is calculated.</p> "
            "<p>Exclude lead-in e.g. 'The minimum payment is calculated as...' and label directly "
            "from e.g. 'the minimum of...'.</p> "
            "<p>Do <em>NOT</em> include clauses from related subjects e.g. how account balance is "
            "calculated</p>"
        ),
    ),
    FieldConfiguration(0, "Local Terms", ignore=True,
        annotation_guidance=(
            "<p>Label full terms specific to residents of certain states/countries, or applying "
            "only in particular jurisdictions.</p> "
            "<p><em>Include</em> the scope of where the terms apply e.g. 'Residents of GA and "
            "VA...'</p> "
            "<p><em>Include</em> locally-applicable interest rates, instead of annotating these "
            "with the 'APR - ' classes</p>"
        ),
    )
]
for ix, cfg in enumerate(fields):
    cfg.class_id = ix

# Save the configuration to file:
with open("data/field-config.json", "w") as f:
    f.write(json.dumps(
        [cfg.to_dict() for cfg in fields],
        indent=2,
    ))

# And print out a simple list:
entity_classes = [f.name for f in fields]
print("\n".join(entity_classes))

Agreement Effective Date
APR - Introductory
APR - Balance Transfers
APR - Cash Advances
APR - Purchases
APR - Penalty
APR - General
APR - Other
Fee - Annual
Fee - Balance Transfer
Fee - Late Payment
Fee - Returned Payment
Fee - Foreign Transaction
Fee - Other
Card Name
Provider Address
Provider Name
Min Payment Calculation
Local Terms


> ▶️ Your field configuration should now be stored in [data/field-config.json](data/field-config.json)

## Data collection

To efficiently annotate training data for entity extraction on documents, we'll want to work **visually**: Highlighting matches, and perhaps also collecting manual transcription reviews - in case we'd like to extend the model later to support correcting text.

[Amazon SageMaker Ground Truth](https://aws.amazon.com/sagemaker/groundtruth/) provides an out-of-the-box annotation UI [for bounding boxes](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-bounding-box.html) which will be useful for this: And can also be incorporated within [customized annotation UIs](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates.html) via the [crowd-bounding-box](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-bounding-box.html) element.

However, at the time of writing, the bounding box annotation tool supports images but not PDFs. Therefore to prepare for data annotation we'll need to:

- Run our documents through Amazon Textract
- Extract individual page images from the PDFs to use through the annotation UI
- Collate the page images and Textract results together, ready for annotation

For a significantly sized corpus like this, we'd also benefit from filtering down the data a little - to save time and cost by Textracting and converting only the amount of data we'll need.

### Filter a sample of the document set

To limit down our input corpus, we'll:

1. Take a random (but reproducible) split of the first N documents

2. ...Ensuring we *include* all documents from the previously annotated sets (otherwise, we won't be able to use these annotations in model training).

> ⚙️ If you're using a custom dataset:
>
> - Remember to **delete the `data/annotations/augmentation-*` folders** before running this to avoid errors. This is because the `list_preannotated_textract_uris()` function below automatically tries to include any already-annotated documents in the sample.
> - The exclusion rules in `include_filename()` may not be relevant to you, so consider reviewing them.

This last point (2) is not as straightforward as you might expect, because in addition to crawling multiple jobs' annotation files:

- The annotation files may contain only the Textract result URI (not the raw document URI - we'll need to map across to that), and
- If some annotation was done on other AWS accounts (like for our sample dataset), the actual bucket name and prefix may not match what you're currently working with.

So we'll first construct that list of which documents were annotated already, which you can inspect and edit if the logic isn't working for your custom datasets:

In [96]:
# Crawl source annotated Textract URIs from the job manifests:
annotated_textract_s3uris = util.ocr.list_preannotated_textract_uris(
    ann_jobs_folder="data/annotations",
    exclude_job_names=["LICENSE"],
)

# List raw document URIs in S3
with open("data/raw-all.manifest.jsonl") as f:
    raw_doc_s3uris = [json.loads(l)["raw-ref"] for l in f]

# Define how to check for matches:
def textract_uri_matches_doc_uri(tex_uri, doc_uri) -> bool:
    """Customize this function if needed for your use case's data layout"""
    # With our sample, Textract URIs will look like:
    # some/prefix/data/textracted/subfolders/file.pdf/consolidated.json
    # (Due to this sample's Textract batch processing logic - will likely be different if you're
    # trying to use previously-Textracted documents).
    tex_s3key = tex_uri[len("s3://"):].partition("/")[2]
    # With our sample, Raw URIs will look like:
    # some/prefix/data/raw/subfolders/file.pdf
    doc_s3key = doc_uri[len("s3://"):].partition("/")[2]

    # Given the expectations above:
    tex_rel_filepath = tex_s3key.partition("data/textracted/")[2].rpartition("/")[0]
    doc_rel_filepath = doc_s3key.partition("data/raw/")[2]
    return doc_rel_filepath == tex_rel_filepath

# Build the list of docs for which some annotations exist (prioritising debug over speed here):
annotated_doc_s3uris = set()
for uri in annotated_textract_s3uris:
    matching_doc_s3uris = [
        doc_s3uri
        for doc_s3uri in raw_doc_s3uris
        if textract_uri_matches_doc_uri(uri, doc_s3uri)
    ]
    n_matches = len(matching_doc_s3uris)
    if n_matches == 0:
        raise ValueError(
            "Couldn't find matching document in dataset for annotated Textract URI: %s"
            % (uri,)
        )
    if n_matches > 1:
        logger.warning(
            "Textract URI matched %s document URIs: Matching criterion may be too loose.\n%s\n%s",
            n_matches,
            uri,
            matching_doc_s3uris,
        )
    annotated_doc_s3uris.update(matching_doc_s3uris)

# This sorted list of required document S3 URIs is the main result you need to get to here:
annotated_doc_s3uris = sorted(annotated_doc_s3uris)
print(f"Found {len(annotated_doc_s3uris)} docs with pre-existing annotations")
print("For example:")
print("\n".join(annotated_doc_s3uris[:5] + ["..."]))

ValueError: Couldn't find matching document in dataset for annotated Textract URI: s3://DOC-EXAMPLE-BUCKET/EXAMPLE-PREFIX/data/textracted/American Airlines Federal Credit Union/Visa Credit Card Agreement and Disclosure.pdf/consolidated.json

With the list of required (already annotated) documents prepared, we're ready to randomly sample a corpus that includes them.

The example Credit Card Agreements dataset includes some **non-English documents**. By default we exclude these (in case you can't read Spanish well enough to annotate them!) ...But you could choose to include them: As the multi-lingual [LayoutXLM model](https://huggingface.co/docs/transformers/model_doc/layoutxlm) has been tested with this sample.

In [104]:
N_DOCS_KEPT = 120
SKIP_SPANISH_DOCS = True


def include_filename(name: str) -> bool:
    """Filter out likely Spanish/non-English docs (if SKIP_SPANISH_DOCS enabled)"""
    if not name:
        return False
    if not SKIP_SPANISH_DOCS:
        return True
    name_l = name.lower()
    if "spanish" in name_l:
        return False
    if re.search(r"espa[nñ]ol", name_l):
        return False
    if "tarjeta" in name_l or re.search(r"cr[eé]dito", name_l):
        return False
    if re.search(r"[\[\(]esp?[\]\)]", name_l):
        return False
    return True


if N_DOCS_KEPT < len(annotated_doc_s3uris):
    raise ValueError(
        "Existing annotations cannot be used for model training unless the target documents are "
        "Textracted. To proceed with fewer docs than have already been annotated, you'll need to "
        "`exclude_job_names` per the 'data/annotations' folder (e.g. ['augmentation-1']) AND "
        "remember to not include them in notebook 2 (model training). Alternatively, increase "
        f"your N_DOCS_KEPT. (Got {N_DOCS_KEPT} vs {len(annotated_doc_s3uris)} prev annotations)."
    )

with open("data/raw-all.manifest.jsonl") as f:
    # First apply filtering rules:
    sampled_docs = [
        doc for doc in (json.loads(line) for line in f)
        if include_filename(doc["raw-ref"])
    ]

# Forcibly including the pre-annotated docs *after* the shuffling ensures that the order of
# sampling new docs is independent of what/how many have been pre-annotated:
required_docs = [d for d in sampled_docs if d["raw-ref"] in annotated_doc_s3uris]
random.Random(1337).shuffle(sampled_docs)
new_docs = [d for d in sampled_docs if d["raw-ref"] not in annotated_doc_s3uris]
sampled_docs = sorted(
    required_docs + new_docs[:N_DOCS_KEPT - len(required_docs)],
    key=lambda doc: doc["raw-ref"],
)

# Write the selected set to file:
with open("data/raw-sample.manifest.jsonl", "w") as f:
    for d in sampled_docs:
        f.write(json.dumps(d) + "\n")

print(f"Extracted random sample of {len(sampled_docs)} docs")
sampled_docs[:5] + ["..."]

Extracted random sample of 100 docs


[{'raw-ref': 's3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/019112540_-_Juul_Labs_-_Francis_Howell_School_District-3.jpeg'},
 {'raw-ref': 's3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/019121420_-_Juul_Labs_-_Bronstein-4.jpeg'},
 {'raw-ref': 's3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/020031085_2020-03-06_-_Juul_Labs-Breathe_DC-3.jpeg'},
 {'raw-ref': 's3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/1150230-3.jpeg'},
 {'raw-ref': 's3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/1150230-4.jpeg'},
 '...']

> ▶️ In [data/raw-sample.manifest.jsonl](data/raw-sample.manifest.jsonl) you should now have an alphabetized list of the `N_DOCS_KEPT` randomly selected documents, which should include any documents referenced in existing annotations under `data/annotations`.

### OCR the input documents

> ⚠️ **Note:** Refer to the [Amazon Textract Pricing Page](https://aws.amazon.com/textract/pricing/) for up-to-date guidance before running large extraction jobs.
>
> At the time of writing, the projected cost (in `us-east-1`, ignoring free tier allowances) of analyzing 100 documents with 10 pages on average was approximately \\$67 with `TABLES` and `FORMS` enabled, or \\$2 without. Across the full corpus, we measured the average number of pages per document at approximately 6.7.

With (a subset of) the raw documents selected, the next ingredient is to link them with Amazon Textract-compatible OCR results in a new manifest - with entries something like:

```json
{"raw-ref": "s3://doc-example-bucket/folder/mydoc.pdf", "textract-ref": "s3://doc-example-bucket/folder/mydoc-textracted.json"}
```

---
⚙️ If you're experimenting with a custom dataset, you might **already have Amazon Textract results** for your documents:

- These must be uploaded to **Amazon S3 in the same AWS Region** you're working in, in a location this SageMaker notebook's execution role has access to.
- They'll need to be under **one shared S3 prefix** for model training later (and training may be more performant/cheaper if that prefix doesn't have too many *other* files under it)
- They must be in an **Amazon Textract output-compatible format**, as the model training script later will assume this.
- Ideally, the results should be **consolidated to a single JSON per raw document** - which is done automatically by the pipeline in this sample, but may not be the case if you used the the Amazon Textract [StartDocumentAnalysis API](https://docs.aws.amazon.com/textract/latest/dg/API_StartDocumentAnalysis.html#Textract-StartDocumentAnalysis-request-OutputConfig) with a direct `OutputConfig`.
    - The reason for this is that we'll construct a *page-level* manifest later, and need to be confident we're linking each page image to the correct corresponding page result from Textract JSON.
    - If your Textract JSON is split into multiple objects per input document and you'd like to avoid re-processing the corpus, you could explore consolidating the JSONs to document level via simple Python. Check that when loaded with [`trp`](https://github.com/aws-samples/amazon-textract-response-parser/blob/master/src-python/README.md), the "pages" in your final file correspond correctly to the source document pages.

The exact correspondence from input document URI to existing OCR result JSON URI will depend on your environment, but here's some example code to show how you might crawl some pre-existing results:

In [100]:
#### EXAMPLE mapping for pre-Textracted documents with existence check:

# Given that raw docs live under some S3 prefix:
raw_s3uri_prefix = raw_s3uri

# ...And Amazon Textract results live under another:
textract_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/textracted"


# ...And you can define a mapping from one to the other:
def doc_uri_to_textract_uri(doc_uri: str) -> str:
    if not doc_uri.startswith(raw_s3uri_prefix):
        raise ValueError(
            "Document S3 URI '%s' did not start with expected prefix: '%s'"
            % (doc_uri, raw_s3uri_prefix)
        )
    # Replace raw prefix with Textract prefix, and add "/consolidated.json" to filename:
    #return textract_s3uri + doc_uri[len(raw_s3uri_prefix):] + "/consolidated.json"
    # 2023-05-23
    return textract_s3uri + doc_uri[len(raw_s3uri_prefix):] 

# Then build up the combined manifest, checking existence for each result:

print("Building manifest...")
with open("data/raw-sample.manifest.jsonl") as fin:
     with open("data/textracted-all.manifest.jsonl", "w") as fout:
         for doc in (json.loads(line) for line in fin):
             textract_uri = doc_uri_to_textract_uri(doc["raw-ref"])
             if not util.s3.s3_object_exists(textract_uri):
                 raise ValueError(
                     "Mapped Textract result URI does not exist in S3.\nFor: %s\nGot: %s"
                     % (doc["raw-ref"], textract_uri)
                 )
             doc["textract-ref"] = textract_uri
             
             fout.write(json.dumps(doc) + "\n")
print("Done!")

Building manifest...
Done!


---
**For documents not yet run through OCR**:

> ⚙️ The code below uses Amazon Textract to extract document text, which is the most accurate and feature-rich choice for languages the Amazon Textract service supports. If you're working with documents in **unsupported languages**, check out the *"Experimenting with alternative OCR engines"* instructions in [Optional Extras.ipynb](Optional%20Extras.ipynb) for an open source-based option. 

We need to be mindful of the service [quotas](https://docs.aws.amazon.com/general/latest/gr/textract.html#limits_textract) when processing large batches of documents with Amazon Textract, to avoid excessive rate limiting and retries. Since an OCR pipeline solution stack is already set up for this sample, you can use just the *Amazon Textract portion of the pipeline* to process the documents in bulk.

> ⏰ This process took about 5 minutes to run against the 120-document sample set in our tests.

In [102]:
textract_results = util.ocr.call_textract(
    textract_sfn_arn=config.plain_textract_sfn_arn,
    # Can instead use raw-all.manifest.jsonl to process whole dataset (see cost note above):
    input_manifest="data/raw-all.manifest.jsonl",
    manifest_raw_field="raw-ref",
    manifest_out_field="textract-ref",
    # Map subpaths of {input_base} to subpaths of {output_base}:
    output_base_s3uri=textract_s3uri,
    input_base_s3uri=raw_s3uri,
    # You can turn on e.g. "TABLES" and "FORMS" features here, but note this could have a
    # significant impact on Textract API costs:
    features=[],
    skip_existing=True,
)

Textracting PDFs...:   0%|          | 0/100 [00:00<?, ?it/s]

Starting jobs...:   0%|          | 0/100 [00:00<?, ?it/s]

Doc failed to process - see results for details: s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/1150230-3.jpeg
Doc failed to process - see results for details: s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/2019112517_-_Juul_Labs_-_McCulloch-3.jpeg
Doc failed to process - see results for details: s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/2019112510_-_Juul_Labs_-_Quercia-3.jpeg
Doc failed to process - see results for details: s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/2019021438cc_2-3.jpeg
Doc failed to process - see results for details: s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/2019112520_-_Juul_Labs_-_Fay-3.jpeg
Doc failed to process - see results for details: s3://bt-digital-bt-labs-internal-200/DynamicTableParser/4_TrainingData/data/raw/2019112516_-_Juul_Labs_-_Oberhauser-3.jpeg
Doc failed to process - see 

> ⚠️ **If you see errors in the output above:**
>
> - Try re-running the cell - Rate limiting can sometimes cause intermittent failures, and the function will skip successfully processed files in repeat runs.
> - Persistent errors (on custom datasets) could be due to malformed files (remove them from the manifest) or very large files (see the [/CUSTOMIZATION_GUIDE.md](../CUSTOMIZATION_GUIDE.md) for tips on re-configuring your pipeline to handle very large documents).

Once the extraction is done, write (only successful items) to a manifest file:

In [103]:
n_success = 0
n_fail = 0
with open("data/textracted-all.manifest.jsonl", "w") as fout:
    for ix, item in enumerate(textract_results):
        if isinstance(item["textract-ref"], str):
            fout.write(json.dumps(item) + "\n")
            n_success += 1
        else:
            if n_fail == 0:
                logger.error("First failure at index %s:\n%s", ix, item["textract-ref"])
            n_fail += 1

print(f"{n_success} of {n_success + n_fail} docs processed successfully")
if n_fail > 0:
    raise ValueError(
        "Are you sure you want to continue? Consider re-trying to process the failed docs"
    )

2023-05-24 11:47:14,960 root [ERROR] First failure at index 0:
{'executionArn': 'arn:aws:states:us-east-1:015943506230:execution:ProcessingPipelineOCRStepTextractStepTextractStateMachineFA5B3847-JnTeNju1dIbg:notebook-2023-05-24-04-11-48-352', 'stateMachineArn': 'arn:aws:states:us-east-1:015943506230:stateMachine:ProcessingPipelineOCRStepTextractStepTextractStateMachineFA5B3847-JnTeNju1dIbg', 'name': 'notebook-2023-05-24-04-11-48-352', 'status': 'FAILED', 'startDate': datetime.datetime(2023, 5, 24, 4, 11, 48, 380000, tzinfo=tzlocal()), 'stopDate': datetime.datetime(2023, 5, 24, 4, 11, 48, 883000, tzinfo=tzlocal()), 'input': '{"Input": {"Bucket": "bt-digital-bt-labs-internal-200", "Key": "DynamicTableParser/4_TrainingData/data/raw/019112540_-_Juul_Labs_-_Francis_Howell_School_District-3.jpeg"}, "Output": {"Features": [], "Key": "DynamicTableParser/4_TrainingData/data/textracted/019112540_-_Juul_Labs_-_Francis_Howell_School_District-3.jpeg/consolidated.json", "Bucket": "bt-digital-bt-labs

0 of 100 docs processed successfully


ValueError: Are you sure you want to continue? Consider re-trying to process the failed docs

---
> ▶️ Regardless of whether you processed documents here, or indexed previously-Textracted results, you should now have a [data/textracted-all.manifest.jsonl](data/textracted-all.manifest.jsonl) JSON-Lines manifest file mapping source documents `raw-ref` to Amazon Textract result JSONs `textract-ref`: Both as `s3://...` URIs.

### Extract clean input images (batch)

To annotate our documents with SageMaker Ground Truth image task UIs, we need **individual page images**, stripped of EXIF rotation metadata (because, at the time of writing, SMGT ignores this rotation for annotation consistency) and converted to compatible formats (since some formats like TIFF are not supported by most browsers).

For large corpora, this process of splitting PDFs and rotating and converting images may require significant resources - but is easy to parallelize.

Therefore instead of pre-processing the raw documents here in the notebook, this is a good use case for a scalable [SageMaker Processing Job](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html).

The job will require a **custom container image**, since the PDF reading tools we use aren't installed by default in pre-built SageMaker containers and aren't `pip install`able. However, the image has already been built and deployed to [Amazon Elastic Container Registry (ECR)](https://aws.amazon.com/ecr/) by the CDK stack (see `preproc_image` in [/pipeline/\_\_init\_\_.py](../pipeline/__init__.py)). All we need to do here is look it up from the stack parameters:

In [55]:
from sagemaker.processing import FrameworkProcessor

ecr_image_uri = config.preproc_image_uri
print(f"Using pre-built custom container image:\n{ecr_image_uri}")

Using pre-built custom container image:
015943506230.dkr.ecr.us-east-1.amazonaws.com/sm-ocr-preprocs:pytorch-1.10-inf-cpu


Next, we need to **define the inputs and output locations** for the processing job.

In [56]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Output S3 locations:
imgs_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/imgs-clean"
thumbs_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/thumbnails"

Two options are shown below:

- **Option 1:** (Commented-out) To process the whole `data/raw` corpus on S3, you could simply pass the entire prefix as input to the processing job. For large corpora, you'll probably want to scale up the job's compute resources below to complete the work quickly.
- **Option 2:** (Sample default) To process just the sampled subset of files, we'll create an **input manifest file** listing the documents we want.

In [ ]:
#### OPTION 1: For processing the whole raw_s3uri prefix:

# preproc_inputs = [
#     ProcessingInput(
#         destination="/opt/ml/processing/input/raw",  # Expected input location, per our script
#         input_name="raw",
#         s3_data_distribution_type="ShardedByS3Key",  # Distribute between instances, if multiple
#         source=raw_s3uri,  # S3 prefix for full raw document collection
#     ),
# ]
# print("Selected whole corpus")
#### END OPTION 1

> **Note:** The 'Non-augmented' manifest files used below for job data loading are still JSON-based, but a different format from the JSON-**Lines** manifests we use in most other places of this sample. You can find guidance on the [S3DataSource API doc](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html) for manifests as used here, and separate information in the [Ground Truth documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-input-data-input-manifest.html) on the "augmented" JSON-Lines manifests used elsewhere.

In [57]:
#### OPTION 2: For processing the sampled subset of raw docs only:

# Note your source documents must still be under one common S3 prefix, and the folder structure
# under this will be mapped through to the outputs:
raw_s3uri_prefix = raw_s3uri  # (*No* trailing slash)

# Load the list of docs from file and add final filters:
with open("data/raw-sample.manifest.jsonl") as fin:
    doc_relpaths = []
    for item in (json.loads(line) for line in fin):
        if item["raw-ref"].startswith(raw_s3uri_prefix + "/"):
            doc_relpaths.append(item["raw-ref"][len(raw_s3uri_prefix) + 1:])
        else:
            raise ValueError(
                "All document S3 URIs for processing job must start with shared S3 prefix %s:\n"
                "%s" % (raw_s3uri_prefix, item["raw-ref"])
            )

# Prepare a true JSON (*NON-JSONLINES*) manifest file for SageMaker Processing:
os.makedirs("data/manifests", exist_ok=True)
preproc_input_manifest_path = "data/manifests/raw-dataclean-input.manifest.json"
with open(preproc_input_manifest_path, "w") as fout:
    fout.write(json.dumps(
        [{ "prefix": raw_s3uri_prefix + "/" }]
        + doc_relpaths
    ))

# Upload the manifest to S3:
preproc_input_manifest_s3uri = f"s3://{bucket_name}/{bucket_prefix}{preproc_input_manifest_path}"
!aws s3 cp {preproc_input_manifest_path} {preproc_input_manifest_s3uri}

# Set the processing job inputs to reference the manifest:
preproc_inputs = [
    ProcessingInput(
        destination="/opt/ml/processing/input/raw",  # Expected input location, per our script
        input_name="raw",
        s3_data_distribution_type="ShardedByS3Key",  # Distribute between instances, if multiple
        s3_data_type="ManifestFile",
        source=preproc_input_manifest_s3uri,  # Manifest of sample raw documents
    ),
]
print("Selected sample subset of documents")
#### END OPTION 2

                                                              
Selected sample subset of documents


The script we'll be using to process the documents is in the same folder as the Dockerfile used earlier to build the container image: [preproc/inference.py](preproc/inference.py).

The code parallelizes processing across available CPUs, and the `ShardedByS3Key` setting used on our [ProcessingInput](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ProcessingInput) above distributes documents between instances if multiple are provided. This means you should be able to adjust both `instance_type` and `instance_count` below, and still take advantage of the resources configured. The process is typically CPU-bound, so the `ml.c*` families are likely a good fit.

The cell below will **run the processing job** and show logs from the job as it progresses. You can also check up on the status and history of jobs in the [Processing page of the Amazon SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/processing-jobs).

> ⏰ **Note:** In our tests, it took (including job start-up overheads):
>
> - About 10 minutes to process the 120-document sample with 2x `ml.c5.xlarge` instances
> - About 17 minutes to process the full 2,541-document credit cards corpus with 5x `ml.c5.4xlarge` instances.

In [58]:
%%time

processor = FrameworkProcessor(
    # Because the custom image already exists, we can just use generic Framework class rather than
    # configuring PyTorchProcessor and worrying about whether PyTorch/Python versions match:
    estimator_cls=util.preproc.DummyFramework,
    image_uri=ecr_image_uri,
    framework_version="",

    base_job_name="ocr-img-dataclean",
    role=sagemaker.get_execution_role(),
    instance_count=2,
    instance_type="ml.c5.xlarge",
    volume_size_in_gb=15,
    max_runtime_in_seconds=60*60,
)

processor.run(
    code="preproc.py",  # PDF splitting / image conversion script
    source_dir="preproc",
    inputs=preproc_inputs[:],  # Either whole corpus or sample, as above
    outputs=[
        ProcessingOutput(
            destination=imgs_s3uri,
            output_name="imgs-clean",
            s3_upload_mode="Continuous",
            source="/opt/ml/processing/output/imgs-clean",  # Output folder, per our script
        ),
        ProcessingOutput(
            destination=thumbs_s3uri,
            output_name="thumbnails",
            s3_upload_mode="Continuous",
            source="/opt/ml/processing/output/thumbnails",  # Resized output for visual models
        ),
    ],
)

...........................2023-05-19 21:33:45,915 [preproc] INFO Parsed job args: Namespace(input='/opt/ml/processing/input/raw', n_workers=4, output='/opt/ml/processing/output/imgs-clean', thumbnails='/opt/ml/processing/output/thumbnails')
2023-05-19 21:33:45,915 [preproc] INFO Additional thumbnail output is ENABLED
2023-05-19 21:33:45,915 [preproc] INFO Reading raw files from /opt/ml/processing/input/raw
2023-05-19 21:33:45,916 [preproc] INFO Processing 60 files across 4 processes
2023-05-19 21:33:49,944 [preproc] INFO Processed doc 1 of 60
2023-05-19 21:33:45,951 [preproc] INFO Parsed job args: Namespace(input='/opt/ml/processing/input/raw', n_workers=4, output='/opt/ml/processing/output/imgs-clean', thumbnails='/opt/ml/processing/output/thumbnails')
2023-05-19 21:33:45,951 [preproc] INFO Additional thumbnail output is ENABLED
2023-05-19 21:33:45,951 [preproc] INFO Reading raw files from /opt/ml/processing/input/raw
2023-05-19 21:33:45,952 [preproc] INFO Processing 60 files across 

Once the images have been extracted, we'll also **optionally** download them locally to the notebook for use in visualizations later:

In [60]:
print(f"Downloading cleaned images from {imgs_s3uri}...")
!aws s3 sync --quiet {imgs_s3uri} data/imgs-clean
print("Done")

Done


You'll see that this job also generates uniformly resized "thumbnail" images per page when the second (optional) `thumbnails` output is specified. These aren't important for the human annotation process, but will be needed later assuming you train a model like [LayoutLMv2](https://huggingface.co/docs/transformers/model_doc/layoutlmv2#transformers.LayoutLMv2Model) which accepts page image inputs.

The older [LayoutLM (v1)](https://huggingface.co/docs/transformers/model_doc/layoutlm#transformers.LayoutLMModel) model doesn't use these.

In [61]:
# You could also download the thumbnail images to see what they look like - but we won't need them:
#!aws s3 sync --quiet {thumbs_s3uri} data/imgs-thumb

### Collate OCR and image data for annotation

Now we have a filtered corpus of documents with Amazon Textract results, plus cleaned and standardized images for each page - all available on Amazon S3.

To prepare for data annotation and later model training, we'll need to collate these together with a **page-level manifest** in JSON-lines format, with records something like:

```json
{"source-ref": "s3://doc-example-bucket/img-prefix/folder/filename-0001-01.png", "textract-ref": "s3://doc-example-bucket/tex-prefix/folder/filename.pdf/consolidated.json", "page-num": 1}
```

Key features of the format are:
- The `source-ref` is the path to a full-resolution cleaned page image (**not** a thumbnail), **but** model training in the next notebook will assume the equivalent thumbnail path is identical, except for some different s3://... bucket & prefix.
- The `page-num` is one-based (always >= 1), and for model training must match the image to the appropriate page number **in the linked Textract JSON file**.
    - For example if you have thumbnail `filename-0001-15.png` for page 15 of some long document, but for some reason your `textract-ref` JSON file contains *only* detections from page 15 of the document, you would set `"page-num": 1`.
- Mapping through the `raw-ref` here is nice to have, but optional, as the model training won't refer to the original document.

> ⚙️ **For custom data setups:** The sample code in this section should work fine if you've used this sample to Textract and clean images for your documents, but if you have a customized setup, you might need to adjust the logic that maps between document, Textract JSON, and page image S3 URIs, to suit your environment.

The key goal is to create a page-level catalogue that we're confident is correct, and for that reason the example function below will actually **validate that the artifacts are present on S3** in the expected locations.

> ⏰ Because of these validation checks, the cell below may a minute or two to run against our 120-document sample set.

In [62]:
warnings = util.preproc.collate_data_manifest(
    # Output file:
    "data/pages-all-sample.manifest.jsonl",
    # Input manifest:
    input_manifest="data/textracted-all.manifest.jsonl",
    # s3://... base URI used to try and map 'textract-ref's to cleaned images:
    textract_s3_prefix=textract_s3uri,
    # The s3://... base URI under which page images are stored:
    imgs_s3_prefix=imgs_s3uri,
    # Optional s3://... base URI also used to try and map 'raw-ref's to images if present:
    raw_s3_prefix=raw_s3uri,
    # Other output manifest settings:
    by="page",
    no_content="omit",
)

if len(warnings):
    raise ValueError(
        "Manifest usable but incomplete - %s docs failed. Please see `warnings` for details"
        % len(warnings)
    )

Building data manifest...:   0%|          | 0/120 [00:00<?, ?it/s]

> ▶️ Whether you were able to use the above code as-is or had to modify it, you should now have a page-level catalogue linking `source-ref`, `textract-ref`, `page-num` in [data/pages-all-sample.manifest.jsonl](data/pages-all-sample.manifest.jsonl)

Let's briefly explore the catalogue we've created.

Each line of the file is a JSON record identifying a particular page:

In [63]:
with open("data/pages-all-sample.manifest.jsonl", "r") as f:
    for ix, line in enumerate(f):
        print(line, end="")
        if ix >= 2:
            print("...")
            break

{"raw-ref": "s3://sagemaker-us-east-1-015943506230/textract-transformers/data/raw/ABNB Federal Credit Union/Visa Platinum Preferred - ABNB Black - Application and Solicitation Disclosure.pdf", "textract-ref": "s3://sagemaker-us-east-1-015943506230/textract-transformers/data/textracted/ABNB Federal Credit Union/Visa Platinum Preferred - ABNB Black - Application and Solicitation Disclosure.pdf/consolidated.json", "source-ref": "s3://sagemaker-us-east-1-015943506230/textract-transformers/data/imgs-clean/ABNB Federal Credit Union/Visa Platinum Preferred - ABNB Black - Application and Solicitation Disclosure-0001-1.png", "page-num": 1}
{"raw-ref": "s3://sagemaker-us-east-1-015943506230/textract-transformers/data/raw/ABNB Federal Credit Union/Visa Platinum Preferred - ABNB Black - Application and Solicitation Disclosure.pdf", "textract-ref": "s3://sagemaker-us-east-1-015943506230/textract-transformers/data/textracted/ABNB Federal Credit Union/Visa Platinum Preferred - ABNB Black - Applicatio

The credit cards corpus has a very skewed distribution of number of pages per document, with a few outliers dragging up the average significantly. In our tests on corpus-wide statistics:

- The overall average was **~6.7 pages per document**
- The 25th percentile was 3 pages; the 50th percentile was 6 pages; and the 75th percentile was 11 pages
- The longest document was 402 pages

Your results for sub-sampled sets will likely vary a little - but can be analyzed as below:

In [64]:
with open("data/pages-all-sample.manifest.jsonl", "r") as f:
    manifest_df = pd.DataFrame([json.loads(line) for line in f])
page_counts_by_doc = manifest_df.groupby("textract-ref")["textract-ref"].count()

print("Document page count statistics")
page_counts_by_doc.describe()

Document page count statistics


count    120.000000
mean       9.716667
std        7.508184
min        1.000000
25%        4.000000
50%        6.000000
75%       18.000000
max       31.000000
Name: textract-ref, dtype: float64

## Annotation infrastructure

To create a labelling job in Amazon SageMaker Ground Truth, we'll need to specify

- **Who's** doing the labelling - which could be your own internal teams, the public crowd via Amazon Mechanical Turk, or skilled workers supplied by vendors through the AWS Marketplace
- **What** the task will look like - which could be using the [built-in task UIs](https://docs.aws.amazon.com/sagemaker/latest/dg/sms.html) or [custom workflows](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates.html).
- **Where** the input data sourced from and the results will be saved to (locations on Amazon S3)

### Create a private workteam

For this demo, you'll set up a private work "team" for just yourself to test out the annotation process.

▶️ **Open** the [Amazon SageMaker Ground Truth console, *Labeling Workforces* page](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-workforces)

> ⚠️ **Check** SM Ground Truth opens in the same **AWS Region** where this notebook and your CloudFormation stack are deployed: You may find it defaults to `N. Virginia`. Use the drop-down in the top right of the screen to switch regions.

▶️ **Select** the *Private* tab and click **Create private team**

- Choose an appropriate **name** for your team e.g. `just-me`
- (If you get the option) select to **Invite new workers via email** and enter your email address (you'll need access to this address to log in and annotate the data)
- And leave the other (Cognito, SNS, etc) parameters as default.

▶️ **If you didn't get the option** to add workers during team creation (typically because your account is already set up for SageMaker Ground Truth), then after the team is created you can:

- Click **Invite new workers** to add your email address to the workforce, and then
- Click on your **team name** to open the team details, then navigate to the *Workers tab* to add yourself to the team

▶️ **Copy** the *name* of your workteam and paste it into the cell below, to store it:

In [ ]:
workteam_name = "just-me"  # TODO: Update this to match yours, if different

workteam_arn = util.smgt.workteam_arn_from_name(workteam_name)

Finally:

▶️ **Check your email** for an invitation and log in to the labelling portal. You'll be asked to configure a password on first login.


Your completed setup should look something like this in the AWS Console:

![](img/smgt-private-workforce.png "Screenshot of SageMaker Ground Truth private workforces configuration")

### Set up the custom task template

This sample provides 2 options for data annotation:

1. Use the **built-in [Bounding Box tool](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-bounding-box.html)**
2. Use the provided **custom task template** which collects transcription reviews as well as bounding boxes

We recommend **experimenting with the custom template** here in the notebook, to get a better understanding of how the model will "see" and use your annotations (and how you might extend this sample for your own use cases).

However, you'll probably want to **use the built-in boxes tool** for the bulk of your annotating work because:

- The ML model we present (in the next notebook) only supports tagging and cannot be directly trained on the text corrections you collect in the custom template
- ...And reviewing the text transcription takes extra time & effort

As detailed [in the developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates.html), custom Ground Truth UIs are HTML [Liquid templates](https://shopify.github.io/liquid/basics/introduction/). You can use the [Crowd HTML Elements](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-reference.html) to embed standard components, but also include custom HTML/CSS/JS as needed. A set of examples is provided in the [amazon-sagemaker-ground-truth-task-uis repository on GitHub](https://github.com/aws-samples/amazon-sagemaker-ground-truth-task-uis).

Since spinning up a labelling job each time to test and debug a custom template would slow down development, SageMaker provides a [RenderUiTemplate API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_RenderUiTemplate.html) for previewing the worker experience.

First, we'll populate the master template `*.liquid.tpl.html` with the entity/field types we configured earlier (and some other automated content) to produce the final SageMaker Ground Truth template `*.liquid.html`:

In [66]:
from bs4 import BeautifulSoup

with open("annotation/ocr-bbox-and-validation.liquid.tpl.html", "r") as ftpl:
    with open("annotation/ocr-bbox-and-validation.liquid.html", "w") as fout:
        template = BeautifulSoup(ftpl.read())

        annotator_el = template.find(id="annotator")
        annotator_el["header"] = "Highlight entities and review their OCR results."
        annotator_el["labels"] = json.dumps(entity_classes)

        if any(f.annotation_guidance for f in fields):
            full_instructions_el = template.find("full-instructions")
            full_instructions_el.append(
                BeautifulSoup(
                    "\n".join(
                        ["<h3>Per-Field Guidance</h3>"]
                        + [
                            f"<h4>{f.name}</h4>\n{f.annotation_guidance}"
                            for f in fields if f.annotation_guidance
                        ]
                    )
                )
            )

        fout.write(template.prettify())

To be able to serve our example images through the UI, SageMaker Ground Truth requires the target S3 bucket to be set up [with CORS permissions](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-cors-update.html) (which is not the same as making the bucket or its contents public).

The cell below will ensure these permissions are set on the bucket configured earlier by `bucket_name`:

In [67]:
util.smgt.ensure_bucket_cors(bucket_name)

Now, we're ready to render the preview of what this task would look like with an actual record from the data manifest:

In [68]:
role = sagemaker.get_execution_role()
smclient = boto3.client("sagemaker")

# Fetch an example record from the manifest:
ix_example = 0
with open("data/pages-all-sample.manifest.jsonl", "r") as fmanifest:
    sample_task_str = None
    for ix, line in enumerate(fmanifest):
        if ix == ix_example:
            sample_task_str = line
            break

# Render the template with the example record:
ui_render_file = "annotation/render.tmp.html"
with open("annotation/ocr-bbox-and-validation.liquid.html", "r") as fui:
    with open(ui_render_file, "w") as frender:
        ui_render_resp = smclient.render_ui_template(
            UiTemplate={"Content": fui.read()},
            Task={"Input": sample_task_str},
            RoleArn=role,
        )
        frender.write(ui_render_resp["RenderedContent"])

print(f"▶️ Open {ui_render_file} and click 'Trust HTML' to see the UI in action!")

▶️ Open annotation/render.tmp.html and click 'Trust HTML' to see the UI in action!


Opening [annotation/render.tmp.html](annotation/render.tmp.html) and clicking **Trust HTML** in the toolbar, you should see a view something similar to the below:

> ℹ️ **Note:** In this task template, you need to click the "Instructions" button to expand the transcription review pane on the left!

![](img/smgt-custom-template-demo.png "Screenshot of custom annotation UI")

Note that:

- When you draw a bounding box on the page image, a new OCR result is populated in the left sidebar prompting you to review (and if necessary, correct) Textract's transcription of the text in that region.
- Overlapping bounding boxes of the same type are combined, allowing you to highlight non-square regions of text.
- Transcription review fields are mandatory: The template should not let you submit the result until all transcriptions have been reviewed in the sidebar.

You should aim to follow the same overlapping conventions when annotating the sample data, even with the built-in task type. Under the hood, the ML model code applies similar logic to map your bounding box annotations to the Textract detected `WORD`s and `LINE`s.

Since reviewing transcriptions makes labelling take longer, you'll probably want to stick to the built-in task template instead of this custom one, unless you have a use case for collecting the text data. The [Exploring sequence-to-sequence models section](Optional%20Extras.ipynb#Exploring-sequence-to-sequence-models) of the **Optional Extras** notebook discusses training generative models to actually normalize and correct OCR transcriptions.

## Annotate data

> ⏰ **If you're short on time**: You can skip the remaining steps in this notebook altogether.
>
> We've provided pre-prepared annotations for 100 pages in the `data/annotations` folder, to augment your work and help train an effective model faster. If you need, you can skip along to the next notebook and select **only** the `augmentation-*` datasets instead of labeling your own too. If you choose to do this, your model will likely be less accurate.

We're now ready to start annotating data, and will typically **iterate over multiple jobs** in this step to start small and then boost model accuracy.

To make incrementally adding to the dataset easy, we'll need to pay particular attention to:

- How we sample data for jobs, with good randomness but no repetition of previously-annotated pages
- How we collect our results to a single consolidated dataset

So let's follow through the steps:

### Collect a dataset

Here we will:

- **Shuffle** our data (in a *reproducible*/deterministic way), to ensure we annotate documents/pages from a range of providers - not just concentrating on the first provider/doc(s)
- **Exclude** any examples for which the page image has **already been labeled** in the `data/annotations` output folder
- **Stratify** the sample, to obtain a specific (boosted) proportion of first-page samples, since we observed the first pages of documents to often be most useful for the fields of interest in the sample credit cards use case. (Many documents use the first page for a fact-sheet/summary, followed by subsequent pages of dense legal terms).

> ⚙️ If you're using a custom dataset:
>
> `select_examples()` below implements a strategy of randomly selecting pages with a fixed, boosted proportion of "page 1s". This might not be appropriate for your use case: For example if you already have a specific planned set of target pages for an extremely small-scale PoC, or if first-page samples are no more relevant than other pages for your documents. Consider revising `select_examples()` to adjust how you select pages for annotation.

In [ ]:
# This cell just defines the necessary functions & constants:

# Keep this the same across the jobs:
annotations_base_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/annotations"


def rel_path_from_s3uri(uri, key_base="data/imgs-clean/") -> str:
    """Extract e.g. 'subfolders/file' from 's3://bucket/.../{key_base}subfolders/file'"""
    return uri[len("s3://"):].partition("/")[2].partition(key_base)[2]


def get_preannotated_imgs(exclude_job_names=[]) -> set:
    """Find the set of relative image paths that have already been annotated"""
    filepaths = set()  # Protect against introducing duplicates
    for job_folder in os.listdir("data/annotations"):
        if job_folder in exclude_job_names:
            logger.info(f"Skipping excluded job {job_folder}")
            continue
        manifest_file = os.path.join(
            "data",
            "annotations",
            job_folder,
            "manifests",
            "output",
            "output.manifest",
        )
        if not os.path.isfile(manifest_file):
            if os.path.isdir(os.path.join("data", "annotations", job_folder)):
                logger.warning(f"Skipping job {job_folder}: No output manifest at {manifest_file}")
            continue
        with open(manifest_file, "r") as f:
            filepaths.update([
                rel_path_from_s3uri(json.loads(line)["source-ref"])
                for line in f
            ])
    return filepaths


def select_examples(
    job_page_count,
    exclude_img_paths=set(),
    job_first_page_pct=0.4,
):
    with open("data/pages-all-sample.manifest.jsonl", "r") as fmanifest:
        examples_all = [json.loads(line) for line in fmanifest]

    # Separate and shuffle the first vs non-first pages:
    examples_all_arefirsts = [item["page-num"] == 1 for item in examples_all]

    examples_firsts = [e for ix, e in enumerate(examples_all) if examples_all_arefirsts[ix]]
    examples_nonfirsts = [e for ix, e in enumerate(examples_all) if not examples_all_arefirsts[ix]]
    random.Random(1337).shuffle(examples_firsts)
    random.Random(1337).shuffle(examples_nonfirsts)

    # Exclude already-annotated images:
    filtered_firsts = [
        e for e in examples_firsts
        if rel_path_from_s3uri(e["source-ref"]) not in exclude_img_paths
    ]
    filtered_nonfirsts = [
        e for e in examples_nonfirsts
        if rel_path_from_s3uri(e["source-ref"]) not in exclude_img_paths
    ]
    print(
        "Excluded %s first and %s non-first pages"
        % (
            len(examples_firsts) - len(filtered_firsts),
            len(examples_nonfirsts) - len(filtered_nonfirsts),
        )
    )

    # Draw from the filtered shuffled lists:
    n_first_pages = round(job_first_page_pct * job_page_count)
    n_nonfirst_pages = job_page_count - n_first_pages
    if n_first_pages > len(filtered_firsts):
        raise ValueError(
            "Unable to find enough first-page records to build manifest: Wanted "
            "%s, but only %s available from list after exclusions (%s before)"
            % (n_first_pages, len(filtered_firsts), len(examples_firsts))
        )
    if n_nonfirst_pages > len(filtered_nonfirsts):
        raise ValueError(
            "Unable to find enough non-first-page records to build manifest: Wanted "
            "%s, but only %s available from list after exclusions (%s before)"
            % (n_nonfirst_pages, len(filtered_nonfirsts), len(examples_nonfirsts))
        )
    print(f"Taking {n_first_pages} first pages and {n_nonfirst_pages} non-first pages.")
    selected = filtered_firsts[:n_first_pages] + filtered_nonfirsts[:n_nonfirst_pages]
    random.Random(1337).shuffle(selected)  # Shuffle again to avoid putting all 1stP at front
    return selected

With this incremental sampling procedure set up, to actually generate a new job input manifest, you just need to specify:

- A unique name for the job
- The number of examples (pages) you'll annotate
- The ratio of first-pages to non-first pages (e.g. 0.4 -> 40% of examples will be the first page of a document)

Run the cells below to create a new labelling job input manifest and then upload it to Amazon S3.

> ⚠️ **Warning:** If you've just completed an annotation job below, make sure you've `s3 sync`ed results back to the `data/annotations` folder - otherwise you'll set up a new job for the same pages again!

In [ ]:
annotation_job_name = "cfpb-boxes-1"  # What will this job be called?
job_page_count = 20  # How many pages will we annotate?
job_first_page_pct = .4  # What proportion of pages should be first pages of a doc?

preannotated_img_paths = get_preannotated_imgs()
input_manifest_file = f"data/manifests/{annotation_job_name}.jsonl"
os.makedirs("data/manifests", exist_ok=True)
print(f"'{annotation_job_name}' saving to: {input_manifest_file}")
with open(input_manifest_file, "w") as f:
    for ix, example in enumerate(select_examples(
        job_page_count,
        exclude_img_paths=preannotated_img_paths,
        job_first_page_pct=job_first_page_pct,
    )):
        if ix < 3:
            print(example)
        elif ix == 3:
            print("...")
        f.write(json.dumps(example) + "\n")

In [ ]:
input_manifest_s3uri = f"s3://{bucket_name}/{bucket_prefix}{input_manifest_file}"
!aws s3 cp $input_manifest_file $input_manifest_s3uri

> ▶️ Above we created another manifest file matching the `source-ref`, `textract-ref`, `page-num` structure from before... But **only** listing the subset of pages to be annotated in this particular labelling job.

### Create the labelling job

To minimize the risk of errors and get started quickly, you're recommended to create your labeling job by running the utility function provided below.

This will set up a job with the **default pre-built bounding box** template by default, for faster labelling. If you want to collect OCR transcription reviews as well (for example, to measure OCR accuracy on your fields of interest or train text normalizing models), you can un-comment and fill in the extra optional arguments to use the custom template we saw earlier.

In [ ]:
print(f"Starting labeling job {annotation_job_name}\non data {input_manifest_s3uri}\n")
create_labeling_job_resp = util.smgt.create_bbox_labeling_job(
    annotation_job_name,
    bucket_name=bucket_name,
    execution_role_arn=role,
    fields=fields,
    input_manifest_s3uri=input_manifest_s3uri,
    output_s3uri=annotations_base_s3uri,
    workteam_arn=workteam_arn,
    s3_inputs_prefix=f"{bucket_prefix}data/manifests",

    # To create a review/adjustment job from a manifest with existing labels in:
    # reviewing_attribute_name="label",

    # To use the custom task template (adding transcription review):
    # task_template="annotation/ocr-bbox-and-validation.liquid.html",
    # pre_lambda_arn="arn:aws...{AnnotationInfra->PreLabelingFunction Lambda from your CFn stack}",
    # post_lambda_arn="arn:aws...{AnnotationInfra->PostLabelingFunction Lambda from your CFn stack}",
)
print(f"\nLABELLING JOB STARTED:\n{create_labeling_job_resp['LabelingJobArn']}")

Alternatively, you can also explore creating the job through the [AWS Console for SageMaker Ground Truth](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-jobs) (check your AWS Region!) by clicking on *Create labeling job*:

- Leave (as default) the *label attribute name* the same as the *job name*
- Select **Manual data setup** and use:
  - The `input_manifest_s3uri` (`s3://[...].jsonl`) from above for the input location
  - The `annotations_base_s3uri` (`s3://[...]/data/annotations`) with **no trailing slash** for the output location
- Select or create any **SageMaker IAM execution role** that has access to the `bucket_name` we're using.
- For **task type**, select *Image > Bounding Box* (for the default bounding-box UI) or *Custom* (for the custom UI with OCR transcription reviews)
- On the second screen, be sure to use **worker type** *Private* and select the workteam we made earlier from the dropdown.
- For the built-in task type, you'll need to enter the **labels** manually exactly in the order that we defined them in this notebook.
- For the custom task type:
  - Copy the contents of the `*.liquid.html` file above into the *Template body* section (**NOT** the `*.tpl.liquid.html`, which has placeholders e.g. for the list of classes)
  - Select the `PreLabelingFunction` and `PostLabelingFunction` Lambda functions created by the Pipeline CDK stack (you can find these via the [CloudFormation console](https://console.aws.amazon.com/cloudformation/home?#/stacks))

The cell below prints out some of these values to help:

In [ ]:
print(input_manifest_s3uri)
print(annotations_base_s3uri)
print(role)
print("\n".join(["\nLabels:", "-------"] + entity_classes))

### Label the data!

Now that the labeling job has been created, you'll see a new task for your user in the labeling portal (If you lost the portal link from your email, you can access it from the *Private* tab of the [SageMaker Ground Truth Workforces console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-workforces)).

> ⏰ SageMaker Ground Truth processes the job data in batches, so it might take a minute or two for the job to appear in your list.
>
> If it's taking a long time, you can:
>
> - Double-check the job in the [Labeling jobs page of the Console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-jobs) to see if it's failed to start due to some error
> - Check the job is set up for a workteam that you're a member of
> - Check your user is showing as *Verified* and *Enabled* (i.e. that you completed the email verification successfully) in the *Private* tab of the [Workforces console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-workforces)

▶️ Click **Start working** and annotate the examples until the all are finished and you're returned to the portal homepage.

▶️ **Try to be as consistent as possible** in how you annotate the classes, because inconsistent annotations can significantly degrade final model accuracy. Refer to the guidance (in this notebook and the 'Full Instructions') that we applied when annotating the example set.

![](img/smgt-task-pending.png "Screenshot of SMGT labeling portal with pending task")

### Sync the results locally (and iterate?)

Once you've finished annotating and the job shows as "Complete" in the [SMGT Console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-jobs) (which **might take an extra minute or two**, while your annotations are consolidated), you can download the results here to the notebook via the cell below:

In [ ]:
!aws s3 sync --quiet $annotations_base_s3uri ./data/annotations

You should see a subfolder created with the name of your annotation job, under which the **`manifests/output/output.manifest`** file contains the consolidated results of your labelling - again in the open JSON-Lines format.

▶️ **Check** your results appear as expected, and explore the file format.

> Because label outputs are in JSON-Lines, it's easy to consolidate, transform, and manipulate these results as required using open source tools!

If you like, you can expand your dataset with **additional labelling jobs** by repeating these steps from [Collect a dataset](#Collect-a-dataset) down to here.

> ⚠️ Take care to set a different `annotation_job_name` each time, as these must be unique.

## Next Steps

In this notebook we set up the modelling objective, collected the project dataset, and annotated (perhaps multiple) sets of training data.

In the next, we'll consolidate these output manifests (together with some pre-prepared example data) and actually train/deploy our ML model.

So you can now open up **notebook [2. Model Training.ipynb](2.%20Model%20Training.ipynb)**, and follow along!